In [1]:
import os
os.chdir('D:\Deep Learning\Tensorflow & Keras\deploy2\project3')

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [3]:
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_dir: Path
    checkpoint_filepath: Path

In [4]:
from TomatoDiseases.constant import *
from TomatoDiseases.utils.common import read_yaml, create_directories

In [5]:
class ConfigManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        checkpoint_dir = os.path.dirname(config.checkpoint_filepath)
        create_directories([Path(checkpoint_dir),
                            Path(config.tensorboard_dir)])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir= config.root_dir,
            tensorboard_dir= config.tensorboard_dir,
            checkpoint_filepath= config.checkpoint_filepath
        )
        
        return prepare_callbacks_config

In [7]:
import os
import tensorflow as tf
import time

In [8]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def create_tensorboard(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        running_log_dir = os.path.join(self.config.tensorboard_dir, f'tensorboard_logs_at_{timestamp}')
        return tf.keras.callbacks.TensorBoard(log_dir=running_log_dir)
    
    @property
    def create_checkpoint(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= self.config.checkpoint_filepath,
            save_best_only=True
        )
        
    def get_tensorboard_and_checkpoint(self):
        return [self.create_tensorboard, self.create_checkpoint]

In [9]:
try:
    config = ConfigManager()
    get_callbacks_config = config.get_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=get_callbacks_config)
    prepare_callbacks.get_tensorboard_and_checkpoint()
except Exception as e:
    raise e

[2023-07-11 00:23:17,403: INFO: common: yaml file: config.yaml loaded successfully]
[2023-07-11 00:23:17,406: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-11 00:23:17,407: INFO: common: created directory at: artifacts]
[2023-07-11 00:23:17,408: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-07-11 00:23:17,409: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
